Loading necessory packages

In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/trdapps/linux-x86_6

Loading Train dataset ...

In [2]:
df = pd.read_csv('Train.csv')
print('Length of dataset before preprocessing: ', len(df))
print('\n We also have {} different labels/class including {} \n'.format(len(df['Label'].unique()), df['Label'].unique()))

Length of dataset before preprocessing:  1436

 We also have 20 different labels/class including ['POLITICS' 'HEALTH' 'LAW/ORDER' 'RELIGION' 'FARMING'
 'WILDLIFE/ENVIRONMENT' 'SOCIAL ISSUES' 'SOCIAL' 'OPINION/ESSAY'
 'LOCALCHIEFS' 'WITCHCRAFT' 'ECONOMY' 'SPORTS' 'RELATIONSHIPS' 'TRANSPORT'
 'CULTURE' 'EDUCATION' 'MUSIC' 'ARTS AND CRAFTS' 'FLOODING'] 



In [3]:
print('Each sample in raw dataset include a col that contains 15-16 different sentences...\n')
df.head(1500)

Each sample in raw dataset include a col that contains 15-16 different sentences...



,ID,Text,Label
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER
...,...,...,...
1431,ID_zmTmmEio,Eni Minibus Ati Ali ndi Ufulu Wokweza Mitengo ...,TRANSPORT
1432,ID_znOlIaGQ,Kachali apepesa: Kulankhula motumbwa kuthe An...,POLITICS
1433,ID_znracTjN,Mawu supports non-fiction writers The Malawi ...,EDUCATION
1434,ID_ztdsmmva,Tame Mwawa: Phwete ndiye kudya kwake Sewero l...,SOCIAL ISSUES


Generating new table including refID of news agency, sentence in the news, Label of each sentence

In [4]:
X_data = pd.DataFrame(columns=['refID', 'Sentence', 'Label'])
for idx in range(len(df)):
    tmp = df.loc[idx]['Text'].split('\n')
    for jdx in range(len(tmp)):
        X_data = X_data.append({'refID': df.loc[idx]['ID'],'Sentence': tmp[jdx], 'Label': df.loc[idx]['Label']}, ignore_index=True)

preprocessing all samples by changing them to lowercase and erasing punctuations

In [5]:
print('Applying preprocessing steps : Changing all characters to lower-case and removing punctuation')
X_data.loc[:,"Sentence"] = X_data.Sentence.apply(lambda x : str.lower(x))
X_data.loc[:,"Label"] = X_data.Label.apply(lambda x : str.lower(x))
X_data.loc[:,"Sentence"] = X_data.Sentence.str.replace('[^\w\s]','')
X_data.loc[:,"Sentence"] = X_data.Sentence.apply(lambda x : x.translate(string.punctuation))

Applying preprocessing steps : Changing all characters to lower-case and removing punctuation


changing Labels to indecies ...

In [6]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
X_data.loc[:,"Label"] = encoder.fit_transform(X_data.Label)

In [7]:
X_data

,refID,Sentence,Label
0,ID_AASHwXxg,mwangonde khansala wachinyamata akamati achin...,11
1,ID_AASHwXxg,mbiri ya maphunziro anu ndi yotani maphunziro...,11
2,ID_AASHwXxg,mudayamba bwanji zandale kuyambira ndili wach...,11
3,ID_AASHwXxg,ntchito mukugwira ndi zomwe munkayembekezera ...,11
4,ID_AASHwXxg,masomphenya anu ndi otani pandale ine ndine m...,11
...,...,...,...
16496,ID_zteydTpN,phiri adati zomwe adakambirana akuluakuluwo a...,11
16497,ID_zteydTpN,mmbuyomu kafukufuku yemwe nyuzipepala ya the ...,11
16498,ID_zteydTpN,mchaka cha 2006 mtsogoleri wa dziko lino pete...,11
16499,ID_zteydTpN,pamsonkhanowo padali a zipani zosiyanasiyana ...,11


split the dataset into training and validation datasets 

In [8]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X_data['Sentence'], X_data['Label'])

approach 1 : encoding words using raw term count features for LDA.

In [9]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data['Sentence'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

approach 2 : encoding words using rExtracting tf-idf features for NMF

In [10]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(X_data['Sentence'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [11]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(predictions, valid_y)

In [12]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print('NB, Count Vectors: ', accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('NB, N-Gram Vectors: ', accuracy)


NB, Count Vectors:  0.4650993698497334
NB, N-Gram Vectors:  0.38342220067862337


In [13]:
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print('SVM, Count Vectors: ', accuracy)

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('SVM, N-Gram Vectors: ', accuracy)

SVM, Count Vectors:  0.2433349491032477
SVM, N-Gram Vectors:  0.2433349491032477


In [14]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print('RF, Count Vectors: ', accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('RF, N-Gram Vectors: ', accuracy)

RF, Count Vectors:  0.422200678623364
RF, N-Gram Vectors:  0.34827920504120213


-

-

-

-

-

In [15]:
# TODO apply deep learning model

In [16]:
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, GlobalMaxPooling1D, Conv1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [17]:
len(X_data['Sentence'].unique()), train_x.shape, valid_x.shape

(14922, (12375,), (4126,))

In [18]:
from nltk.tokenize import word_tokenize
all_words = []
for sent in X_data['Sentence']:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [19]:
unique_words = set(all_words)
print(len(unique_words))

51137


In [20]:
embedded_sentences = [one_hot(sent, 50) for sent in X_data['Sentence']]

In [21]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(X_data['Sentence'], key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [22]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences[0])

[33  2 41 13  7 11 23  7 48 49 47 47 13 11 33 45  6  7  4 25 37 33 42 10
 23 46 35 47 27  6 13 41  8 49 12 20 20 20  7 11 33  2 13 36 40 24 20 37
 43  4 18  4 20 34  9 27 12 49 23 20 35 32 27  8 11  1 33 49 40 37 33 21
 11 11 24 28  6 21 40 33  9  1 40  5  8 37  2 37 45 31 47 20 36 22 40 23
 20 38 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [23]:
embedding_dims, filters, kernel_size, hidden_dims = 50, 256, 5, 64

model = Sequential()
model.add(Embedding(50, 20, input_length=length_long_sentence))
model.add(Dropout(0.2)) # masks various input values
    
# Create the convolutional layer
model.add(Conv1D(filters, kernel_size,padding='valid', activation='relu', strides=1))

# Create the convolutional layer
model.add(Conv1D(128, kernel_size,padding='valid', activation='relu', strides=1))

# Create the pooling layer
model.add(GlobalMaxPooling1D())

# Create the fully connected layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# Create the output layer (num_classes)
model.add(Dense(20))
model.add(Activation('softmax'))

# Add optimization method, loss function and optimization value
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt, metrics=['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 472, 20)           1000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 472, 20)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 468, 256)          25856     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 464, 128)          163968    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      

In [24]:
from keras.utils import to_categorical
embeding_labels = to_categorical(X_data['Label'])

In [25]:
embeding_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [26]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(padded_sentences, embeding_labels, shuffle=True)

In [27]:
model.fit(x_train, y_train, batch_size=20, 
          epochs=50, validation_data=(x_valid, y_valid))


score = model.evaluate(x_valid, y_valid, batch_size=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 12375 samples, validate on 4126 samples
Epoch 1/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.6037 - acc: 0.2311 - val_loss: 2.5632 - val_acc: 0.2365
Epoch 2/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.5607 - acc: 0.2373 - val_loss: 2.5282 - val_acc: 0.2365
Epoch 3/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.5143 - acc: 0.2444 - val_loss: 2.4903 - val_acc: 0.2475
Epoch 4/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.4801 - acc: 0.2477 - val_loss: 2.4742 - val_acc: 0.2511
Epoch 5/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.4522 - acc: 0.2553 - val_loss: 2.4722 - val_acc: 0.2494
Epoch 6/50
12375/12375 [==============================] - 28s 2ms/step - loss: 2.4321 - acc: 0.2590 - val_loss: 2.4561 - val_acc: 0.2516
Epoch 7/50
12375/12375 [===========

In [28]:
print('accuracy score is', score)

accuracy score is [3.36317154301409, 0.20407174308786563]


In [29]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True)
for tridx, tsidx in kfold.split(padded_sentences, embeding_labels):
    # "Fit the model" 
    model.fit(padded_sentences[tridx], embeding_labels[tridx], batch_size=20, 
              epochs=10, validation_data=(padded_sentences[tsidx], embeding_labels[tsidx]))

    # Evaluate the trained model
    score = model.evaluate(padded_sentences[tsidx], embeding_labels[tsidx], batch_size=20) 
    print('Score is : ', score)

Train on 14850 samples, validate on 1651 samples
Epoch 1/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.8390 - acc: 0.4648 - val_loss: 1.5307 - val_acc: 0.5936
Epoch 2/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.7841 - acc: 0.4684 - val_loss: 1.5663 - val_acc: 0.5669
Epoch 3/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.7099 - acc: 0.4798 - val_loss: 1.5870 - val_acc: 0.5512
Epoch 4/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.6916 - acc: 0.4780 - val_loss: 1.6279 - val_acc: 0.5391
Epoch 5/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.6626 - acc: 0.4851 - val_loss: 1.6434 - val_acc: 0.5142
Epoch 6/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.6433 - acc: 0.4896 - val_loss: 1.6748 - val_acc: 0.5051
Epoch 7/10
14850/14850 [==============================] - 32s 2ms/step - loss: 1.6080 - acc: 0.4957 - val_loss: 1.7254 - val_acc: